## Notebook setup

In [ ]:
# Note: This is work in progress and not done yet
import os
import io
import warnings
import datetime
import numpy as np
import base64

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import pandas as pd

from IPython.display import HTML

# import sys
# sys.path.append('../../')

from openbb_terminal.api import openbb
from openbb_terminal import api as openbb_extra
from openbb_terminal.helper_classes import TerminalStyle

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

# Detect if prediction capabilities are present. If they are not, disable prediction in the rest of the script
# so that the report can still be generated without prediction results.
predictions = True
try:
    openbb.stocks.pred.models
except Exception as e:
    predictions = False

In [ ]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = openbb_extra.widgets.html_report_stylesheet()
with open("./openbb_terminal/reports/OpenBB_reports_logo.png", "rb") as image_file:
    openbb_image_encoded = base64.b64encode(image_file.read())

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
pair = "SEKCHF"
report_name = ""

In [ ]:
base = pair[:3]
quote = pair[3:]

data = openbb.forex.load(base, quote, "d", "1day",(datetime.datetime.now() - datetime.timedelta(days=4 * 30)).strftime("%Y-%m-%d"))
data = openbb.stocks.process_candle(data)

author = "Fabian Farestam"
report_title = f"INVESTMENT RESEARCH REPORT ON {pair.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

## Data

In [ ]:
fig, (candles) = plt.subplots(nrows=1, ncols=1, figsize=(11, 5), dpi=150)
openbb.forex.candle(data, base, quote, external_axes=[candles], chart=True)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
price_chart = f.getvalue().decode("utf-8")

In [ ]:
quote_data = openbb.forex.quote(base, quote)
quote_data = pd.DataFrame.from_dict(quote_data)
quote_data.index = quote_data.index.to_series().apply(lambda x: x[3:]).values
quote_data = quote_data.iloc[[0, 2, 5, 4, 7, 8]]

In [ ]:
forward_rates = openbb.forex.forward_rates(quote, base)

In [ ]:
df = openbb.common.technical_analysis.rsi(data["Close"])
rsi_value = round(df.values[-1][0], 2)

In [ ]:
result, _, _ = openbb.common.quantitative_analysis.seasonal_decomposition(data["Adj Close"], False)
seasonal = "Time-Series Level is " + str(round(data["Adj Close"].mean(), 2))

Ft = max(0, 1 - np.var(result.resid)) / np.var(result.trend + result.resid)
seasonal += f"<br><br>Strength of Trend: {Ft:.4f}<br>"

Fs = max(0, 1 - np.var(result.resid) / np.var(result.seasonal + result.resid))
seasonal += f"<br>Strength of Seasonality: {Fs:.4f}<br>"

In [ ]:
summary = openbb.common.quantitative_analysis.summary(data)

## Render the report template to a file

In [ ]:
body = ""

img = f'<img src="data:image/png;base64,{openbb_image_encoded.decode()}" alt="OpenBB" style="width:144px;">'
body += openbb_extra.widgets.header(
    img,
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>INVESTMENT RESEARCH REPORT:</b> {pair}",
)

body += openbb_extra.widgets.tablinks(
    [
        "SUMMARY",
        "Overview",
        "Forward Rates",
        "Quantative Analysis"
    ]
)

htmlcode = openbb_extra.widgets.h(3, "KPIs")
htmlcode += openbb_extra.widgets.kpi(
    [30, 70],
    ["RSI level is oversold", "RSI level is normal", "RSI level is overbought"],
    rsi_value,
)
body += openbb_extra.widgets.add_tab("SUMMARY", htmlcode)


htmlcode = openbb_extra.widgets.row([openbb_extra.widgets.h(3, f"{base}/{quote} Chart") + price_chart])
htmlcode += openbb_extra.widgets.row([openbb_extra.widgets.h(3, f"{base}/{quote} Quotes") + quote_data.to_html()])
body += openbb_extra.widgets.add_tab("Overview", htmlcode)
htmlcode = openbb_extra.widgets.row([openbb_extra.widgets.h(3, f"Forward rates for {base}/{quote}") + forward_rates.to_html()])
body += openbb_extra.widgets.add_tab("Forward Rates", htmlcode)

htmlcode = openbb_extra.widgets.row([openbb_extra.widgets.h(3, f"Summary") + summary.to_html()])
htmlcode += openbb_extra.widgets.row([openbb_extra.widgets.h(3, f"Seasonal Decomposition") + seasonal])
body += openbb_extra.widgets.add_tab("Quantative Analysis", htmlcode)

body += openbb_extra.widgets.tab_clickable_evt()

report = openbb_extra.widgets.html_report(
    title=report_name, stylesheet=stylesheet, body=body
)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)